In [1]:
import pandas as pd
import numpy as np
import os
from importlib import reload

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from sklearn.model_selection import train_test_split
import skimage
from skimage import io
from skimage.transform import resize

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

# local imports
import model as _model
import utils as _tools

In [2]:
"""
control the generation of trainning and dev datasets
"""

train, dev, _ = _tools.make_trainning_data(sample=20000, 
                                           return_frames=True, 
                                           state=1729)

print()
print("Trainning label distribution")
print(train['Cardiomegaly'].value_counts(normalize=True, dropna=False))

print()
print("Development label distribution")
print(dev['Cardiomegaly'].value_counts(normalize=True, dropna=False))

sampling 20000 records
train.shape: (15000, 19)
dev.shape: (5000, 19)
valid.shape: (234, 19)
saved: ./train.csv
saved: ./dev.csv
saved: ./valid.csv

Trainning label distribution
 NaN    0.790267
 1.0    0.123733
 0.0    0.049333
-1.0    0.036667
Name: Cardiomegaly, dtype: float64

Development label distribution
 NaN    0.7884
 1.0    0.1248
 0.0    0.0492
-1.0    0.0376
Name: Cardiomegaly, dtype: float64


In [ ]:
"""
build and train the model
"""

reload(_model)

# build the model
resnet = _model.TransferModel()

# train the model
resnet.train()


-------------------------------
Cardiomegaly Model epoch 1/10
Trainning loss: 0.5109 accuracy: 78.55 %
Validation loss: 0.3703 accuracy: 87.52 %

-------------------------------
Cardiomegaly Model epoch 2/10
Trainning loss: 0.4851 accuracy: 79.00 %
Validation loss: 0.3708 accuracy: 87.54 %

-------------------------------
Cardiomegaly Model epoch 3/10
Trainning loss: 0.4443 accuracy: 79.65 %
Validation loss: 0.3772 accuracy: 85.90 %

-------------------------------
Cardiomegaly Model epoch 4/10
Trainning loss: 0.3894 accuracy: 82.04 %
Validation loss: 0.3587 accuracy: 86.46 %

-------------------------------
Cardiomegaly Model epoch 5/10
Trainning loss: 0.3254 accuracy: 85.63 %
Validation loss: 0.3798 accuracy: 87.10 %

-------------------------------
Cardiomegaly Model epoch 6/10


In [ ]:
# get results on dev set
results = resnet.evaluate_model(resnet.best_model, 
                                resnet.dataloader_dev, 
                                resnet.dev_map)
print(results.shape)

# get distributions of true labels
print()
print(results['y_true'].value_counts(normalize=True))

# get distributions of pred labels
print()
print(results['y_pred'].value_counts(normalize=True))

In [ ]:
results.head()

In [ ]:
results['y_prob'].hist(edgecolor='black', bins=30)

In [ ]:
_time = list(range(len(resnet.train_loss_history)))

plt.plot(_time, resnet.train_loss_history, c='red', label="Trainning")
plt.plot(_time, resnet.dev_loss_history, c='green', label="Testing")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Cross Entropy Loss")

In [ ]:
reload(_tools)

res = _tools.get_classification_metrics(results)
res

In [ ]:
# # get results on valid set
# results = resnet.evaluate_model(resnet.best_model, 
#                                 resnet.dataloader_valid, 
#                                 resnet.valid__map)
# print(results.shape)

# # get distributions of true labels
# print()
# print(results['y_true'].value_counts(normalize=True))

# # get distributions of pred labels
# print()
# print(results['y_pred'].value_counts(normalize=True))

# print()
# res = _tools.get_classification_metrics(results)
# res